In [1]:
# -------- Imports --------
import sys
import os
import numpy as np
import scipy.sparse as sp
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from _utility import *


In [2]:
# -------- Parameters --------
# -- Grid parameters --
Nx, Ny  = 200, 200
dx, dy = 1.0, 1.0
psi_len = Nx*Ny + (Nx-1)*Ny + Nx*(Ny-1) # Number of values for the staggered grid

# -- Wave field definition --
# Pressure field (u)
f0 = 5.0            # Central frequency of the Ricker wavelet
x0, y0 = 50.0, 0.0  # Wavelet center
X, Y = np.meshgrid(np.linspace(-1, 1, Nx), np.linspace(-1, 1, Ny))
u0 = Ricker(f0, X, Y, 0, x0/Nx, y0/Ny)

# Particle velocity field x (vx)
v0x = np.zeros((Ny, (Nx-1)))

# Particle velocity field y (vy)
v0y = np.zeros(((Ny-1), Nx))

# Stack the wave field components
phi_0 = np.hstack([u0.flatten(), v0x.flatten(), v0y.flatten()])

# -- Material properties --
# Velocity (c)
c0 = 3
c_model = c0 * np.ones((Ny, Nx))

# Density (rho)
rho0 = 2
rho_model = rho0 * np.ones((Ny, Nx))
rho_stag_x = rho0 * np.ones((Ny, (Nx-1)))
rho_stag_y = rho0 * np.ones(((Ny-1), Nx))

# -- Subspace mask --
mask = sp.diags(np.random.choice([0, 1], size=psi_len, p=[0.5, 0.5]))


In [3]:
# -------- Material Transform (Acoustic) --------
(B, B_sqrt, B_inv, _) = compute_B(c_model, rho_model, rho_stag_x, rho_stag_y)


In [ ]:
# -------- Loss function 1: Subspace Energy (one state) --------
# Normalize the state and transform it to the energy basis (quantum state)
psi_0 = B_sqrt @ phi_0
norm = np.linalg.norm(psi_0)
psi_0 /= norm

# Compute the ground-truth energy (uniform element volume)
EN_GT = (1/2) * np.linalg.norm(B_sqrt @ (mask @ phi_0))**2 * (dx * dy)
print('Ground-Truth Energy:', EN_GT.round(4))

# Define quantum observable
O_EN = mask
print('Hermitian Observable:', not (O_EN - O_EN.conj().T).nnz)

# Compute expectation value (quantum measurement)
E_EN = np.abs(psi_0.conj().T @ O_EN @ psi_0)
print('Expectation Value:', E_EN.round(4))

# Compute the energy loss (post-processing)
EN_QC = (1/2) * E_EN * norm**2 * (dx * dy)
print('Energy Estimate:', EN_QC.round(4))


Ground-Truth Energy: 0.8838
Hermitian Observable: True
Expectation Value: 0.5048
Energy Estimate: 0.8838


In [ ]:
# -------- Loss function 2: Subspace Energy-transformed L2 Distance  (two states) --------
# Add a comparison state
rand = (np.random.rand(psi_len) - 0.5) * 1
phi_c = mask @ (phi_0 + rand)
phi_l2 = np.hstack([phi_0, phi_c])

# Normalize the state and transform it to the energy basis (quantum state)
psi_l2 = sp.block_diag(2*[B_sqrt]) @ phi_l2
norm = np.linalg.norm(psi_l2)
psi_l2 /= norm

# Compute the ground-truth L2 distance
L2_GT = np.linalg.norm(B_sqrt @ (mask @ phi_0) - B_sqrt @ phi_c)
print('Ground-Truth L2 Distance:', L2_GT.round(4))

# Define quantum observable
O_L2 = sp.block_array([[mask, -mask],
                       [-mask, mask]])
print('Hermitian Observable:', not (O_L2 - O_L2.conj().T).nnz)

# Compute expectation value (quantum measurement)
E_L2 = np.abs(psi_l2.conj().T @ O_L2 @ psi_l2)
print('Expectation Value:', E_L2.round(4))

# Compute the l2 loss (post-processing)
L2_QC = np.sqrt(E_L2) * norm
print('L2 Estimate:', L2_QC.round(4))


Ground-Truth L2 Distance: 82.5013
Hermitian Observable: True
Expectation Value: 0.9992
L2 Estimate: 82.5013
